In [1]:
from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

hamiltonian = SparsePauliOp.from_list(
    [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XY", 0.1810)]
)

ansatz = EfficientSU2(2)
optimizer = COBYLA(maxiter=300)
estimator = Estimator()
vqe = VQE(estimator, ansatz, optimizer)

vqe_result = vqe.compute_minimum_eigenvalue(hamiltonian)

print(vqe_result)


{   'aux_operators_evaluated': None,
    'cost_function_evals': 300,
    'eigenvalue': -0.7029303679808687,
    'optimal_circuit': <qiskit.circuit.library.n_local.efficient_su2.EfficientSU2 object at 0x12eebbc50>,
    'optimal_parameters': {   ParameterVectorElement(θ[0]): -2.3720148109611765,
                              ParameterVectorElement(θ[1]): -1.2567787912927977,
                              ParameterVectorElement(θ[2]): -2.1494442248212,
                              ParameterVectorElement(θ[3]): 3.3595259816647243,
                              ParameterVectorElement(θ[4]): 0.1419643661375142,
                              ParameterVectorElement(θ[5]): 1.8498330063239834,
                              ParameterVectorElement(θ[6]): 2.9205804120054033,
                              ParameterVectorElement(θ[7]): -1.7217815924311655,
                              ParameterVectorElement(θ[8]): 1.6100564101895687,
                              ParameterVectorElement(θ[9]): 1.579

In [2]:
from numpy.linalg import eigvalsh
solution_eigenvalue = min(eigvalsh(hamiltonian))
print(solution_eigenvalue)

-0.7029303944595314


In [2]:
from qiskit.circuit import ParameterVector
from qiskit.quantum_info import SparsePauliOp
from qiskit import QuantumCircuit
from qiskit.primitives import Estimator
from scipy.optimize import minimize
import numpy as np

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    result = estimator.run(circuits=ansatz, observables=hamiltonian, parameter_values=params).result()
    energy = result.values[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)

    return energy

pauli_list = ["YZ", "ZI", "ZZ", "XX"]
coeffs = [0.3980, -0.3980, -0.0113, 0.1810]
hamiltonian = SparsePauliOp(pauli_list, coeffs=coeffs)

theta = ParameterVector('θ', 16)
initial_params = 2 * np.pi * np.random.random(16)

qc = QuantumCircuit(2)
print(theta)
# First layer of Ry and Rz gates
qc.ry(theta[0], 0)
qc.ry(theta[1], 1)
qc.rz(theta[2], 0)
qc.rz(theta[3], 1)

# First entangling layer
qc.cx(0, 1)

# Second layer of Ry and Rz gates
qc.ry(theta[4], 0)
qc.ry(theta[5], 1)
qc.rz(theta[6], 0)
qc.rz(theta[7], 1)

# Second entangling layer
qc.cx(0, 1)

# Third layer of Ry gates
qc.ry(theta[8], 0)
qc.ry(theta[9], 1)
qc.rz(theta[10], 0)
qc.rz(theta[11], 1)

# Third entangling layer
qc.cx(0, 1)

# Fourth layer of Ry and Rz gates
qc.ry(theta[12], 0)
qc.ry(theta[13], 1)
qc.rz(theta[14], 0)
qc.rz(theta[15], 1)

# Draw the circuit
qc.draw('mpl')

estimator = Estimator()

res = minimize(
        cost_func,
        initial_params,
        args=(qc, hamiltonian, estimator),
        method="cobyla",
    )

print(res)

θ, ['θ[0]', 'θ[1]', 'θ[2]', 'θ[3]', 'θ[4]', 'θ[5]', 'θ[6]', 'θ[7]', 'θ[8]', 'θ[9]', 'θ[10]', 'θ[11]', 'θ[12]', 'θ[13]', 'θ[14]', 'θ[15]']
 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: -0.7029303883253292
       x: [ 4.759e+00  3.548e+00 ...  2.175e+00  3.858e+00]
    nfev: 452
   maxcv: 0.0
